# 🎯 LSB Temporal Action Detection - Google Colab

Sistema de detección temporal de acciones para Lengua de Señas Boliviana (LSB)

---

## 📋 Antes de empezar:

1. **Configura GPU**: Runtime → Change runtime type → **T4 GPU**
2. **Duración**: ~2-4 horas para entrenamiento completo
3. **Espacio**: ~10-15GB necesarios

---

### ⚙️ Hardware Recomendado:
- **GPU**: T4 (16GB VRAM) ✅ RECOMENDADO
- **RAM**: 12GB High-RAM

**¿Por qué T4 GPU y no TPU?**
- Video Swin Transformer optimizado para CUDA
- PyTorch nativo (TPU requiere JAX/TensorFlow)
- Mejor soporte para transformers y timm
- 16GB VRAM suficiente para videos

---

## 🔧 1. Setup Inicial

In [ ]:
# Verificar GPU disponible
!nvidia-smi

import torch
print(f"\n{'='*60}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
print(f"{'='*60}")

## 📦 2. Clonar Repositorio

In [ ]:
# Clonar repositorio
!git clone https://github.com/borysinho/LSB-Temporal-Action-Detection.git
%cd LSB-Temporal-Action-Detection

# Ver estructura
!ls -la

## 📥 3. Instalar Dependencias

In [ ]:
# Instalar PyTorch con CUDA (optimizado para Colab T4)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
# Instalar resto de dependencias
!pip install -r requirements.txt -q

print("✅ Instalación completada")

## 💾 4. Conectar Google Drive (Opcional pero Recomendado)

Para:
- Guardar checkpoints
- Cargar tus videos
- Persistir resultados

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Crear directorios en Drive
import os
DRIVE_PATH = '/content/drive/MyDrive/LSB_TAD'
os.makedirs(f"{DRIVE_PATH}/checkpoints", exist_ok=True)
os.makedirs(f"{DRIVE_PATH}/logs", exist_ok=True)
os.makedirs(f"{DRIVE_PATH}/data", exist_ok=True)

print(f"✅ Google Drive montado en: {DRIVE_PATH}")

## 📁 5. Preparar Datos

### Opción A: Subir desde tu Drive
Sube tus videos a `MyDrive/LSB_TAD/data/videos/`

**IMPORTANTE**: También debes subir las carpetas de segmentos procesados. Cada video debe tener su carpeta correspondiente con los segmentos:
- Si tu video se llama `ABECEDARIO.mp4`, debe existir `ABECEDARIO_senas/` con los segmentos
- Sube estas carpetas a la misma ubicación: `MyDrive/LSB_TAD/data/videos/`

### Opción B: Descargar desde URL
Si tienes tus datos en la nube

In [ ]:
# Configurar rutas de datos
import yaml
from pathlib import Path

# Opción A: Usar datos desde Google Drive
USE_DRIVE = True

if USE_DRIVE:
    VIDEO_DIR = f"{DRIVE_PATH}/data/videos"
    CHECKPOINT_DIR = f"{DRIVE_PATH}/checkpoints"
    LOG_DIR = f"{DRIVE_PATH}/logs"
else:
    # Opción B: Usar almacenamiento local de Colab (se pierde al desconectar)
    VIDEO_DIR = "/content/data/videos"
    CHECKPOINT_DIR = "/content/checkpoints"
    LOG_DIR = "/content/logs"
    os.makedirs(VIDEO_DIR, exist_ok=True)
    os.makedirs(CHECKPOINT_DIR, exist_ok=True)
    os.makedirs(LOG_DIR, exist_ok=True)

print(f"📂 Videos: {VIDEO_DIR}")
print(f"💾 Checkpoints: {CHECKPOINT_DIR}")
print(f"📊 Logs: {LOG_DIR}")

In [ ]:
# OPCIONAL: Descargar datos de ejemplo (si tienes URL)
# Descomenta y ajusta según tu caso

# !wget -O /tmp/lsb_videos.zip "TU_URL_AQUI"
# !unzip /tmp/lsb_videos.zip -d {VIDEO_DIR}
# !rm /tmp/lsb_videos.zip

print("⚠️ Asegúrate de tener videos en:", VIDEO_DIR)

## 🏷️ 7. Preparar Anotaciones Temporales

Crear archivo de anotaciones con timestamps de señas

**Nota**: Asegúrate de que tanto los videos como sus carpetas de segmentos estén en Drive

In [ ]:
# Paso auxiliar: Crear estructura de segmentos si no existe
import os
from pathlib import Path
import shutil

print("🔧 Verificando y creando estructura de segmentos...")

# Función para crear carpetas de segmentos básicas
def create_segment_structure(videos_dir):
    """Crea carpetas de segmentos vacías para videos que no las tienen."""
    videos_found = 0
    segments_created = 0
    
    for video_file in Path(videos_dir).rglob("*.mp4"):
        video_name = video_file.stem
        video_parent = video_file.parent
        
        # Verificar si existe carpeta de segmentos
        segment_dir = video_parent / f"{video_name}_senas"
        
        if not segment_dir.exists():
            # Crear carpeta vacía
            segment_dir.mkdir(exist_ok=True)
            print(f"📁 Creada carpeta de segmentos: {segment_dir}")
            segments_created += 1
        
        videos_found += 1
        
        # Limitar output para no saturar
        if videos_found % 100 == 0:
            print(f"   Procesados {videos_found} videos...")
    
    print(f"✅ Verificación completada:")
    print(f"   📹 Videos encontrados: {videos_found}")
    print(f"   📁 Carpetas de segmentos creadas: {segments_created}")
    
    if segments_created > 0:
        print("\n⚠️  IMPORTANTE:")
        print("   Se crearon carpetas de segmentos VACÍAS.")
        print("   Debes subir los archivos de segmentos .mp4 a cada carpeta correspondiente.")
        print("   Cada carpeta _senas debe contener los clips individuales de señas.")
        print("\n   Ejemplo:")
        print("   - TOMATE.mp4 → TOMATE_senas/")
        print("     ├── seña_01_frames_0000-0360.mp4")
        print("     ├── seña_02_frames_0361-0720.mp4")
        print("     └── ...")
    
    return videos_found, segments_created

# Ejecutar creación de estructura
videos_count, segments_created = create_segment_structure(VIDEO_DIR)

if segments_created > 0:
    print(f"\n🛑 DETENTE AQUÍ: Sube los segmentos antes de continuar.")
    print("   Una vez que hayas subido todos los archivos de segmentos,")
    print("   ejecuta la siguiente celda.")
else:
    print("\n✅ Estructura completa detectada. Puedes continuar.")

In [ ]:
# Verificar que tenemos videos y detectar si están segmentados
import os
from pathlib import Path
import re

print("🔍 Verificando estructura de datos...")
print(f"📂 Directorio de videos: {VIDEO_DIR}")

# Contar videos
video_count = len(list(Path(VIDEO_DIR).rglob("*.mp4")))
print(f"📹 Videos encontrados: {video_count}")

# Detectar si los videos están pre-segmentados
segmented_videos = []
segment_dirs = []

# Buscar patrones de videos segmentados (ej: ABECEDARIO_001.mp4, ABECEDARIO_002.mp4, etc.)
for video_file in Path(VIDEO_DIR).rglob("*.mp4"):
    video_name = video_file.stem
    # Buscar patrón: nombre_base + _ + números + .mp4
    match = re.match(r'^(.+?)_(\d+)\.mp4$', video_file.name)
    if match:
        base_name = match.group(1)
        segment_num = int(match.group(2))
        segmented_videos.append((base_name, video_file))

# Agrupar por base_name
segmented_groups = {}
for base_name, video_file in segmented_videos:
    if base_name not in segmented_groups:
        segmented_groups[base_name] = []
    segmented_groups[base_name].append(video_file)

# Verificar carpetas de segmentos tradicionales
segment_dirs = list(Path(VIDEO_DIR).rglob("*_senas"))

print(f"📁 Carpetas de segmentos tradicionales: {len(segment_dirs)}")
print(f"🎬 Grupos de videos segmentados detectados: {len(segmented_groups)}")

# Análisis detallado de las carpetas de segmentos
if segment_dirs:
    print("\n📋 Análisis de carpetas de segmentos:")
    empty_dirs = 0
    dirs_with_files = 0
    total_segments = 0

    # Contar TODAS las carpetas, no solo las primeras 10
    for seg_dir in segment_dirs:
        mp4_files = list(seg_dir.glob("*.mp4"))
        if mp4_files:
            dirs_with_files += 1
            total_segments += len(mp4_files)
        else:
            empty_dirs += 1

    # Mostrar detalles de las primeras 10
    print("  Primeras 10 carpetas:")
    for i, seg_dir in enumerate(segment_dirs[:10]):
        mp4_files = list(seg_dir.glob("*.mp4"))
        if mp4_files:
            print(f"  ✓ {seg_dir.name}: {len(mp4_files)} segmentos")
        else:
            print(f"  ✗ {seg_dir.name}: VACÍA")

    if len(segment_dirs) > 10:
        print(f"  ... y {len(segment_dirs) - 10} carpetas más")

    print(f"\n📊 Resumen de segmentos:")
    print(f"   📁 Carpetas con archivos: {dirs_with_files}")
    print(f"   📁 Carpetas vacías: {empty_dirs}")
    print(f"   🎬 Total de segmentos encontrados: {total_segments}")

# Detectar si todos los videos son segmentos individuales
videos_with_segments = 0
videos_without_segments = 0

for video_file in Path(VIDEO_DIR).rglob("*.mp4"):
    video_name = video_file.stem
    video_parent = video_file.parent

    # Verificar si existe carpeta de segmentos
    segment_dir = video_parent / f"{video_name}_senas"
    if segment_dir.exists() and list(segment_dir.glob("*.mp4")):
        videos_with_segments += 1
    else:
        videos_without_segments += 1

if video_count == 0:
    print("❌ ERROR: No se encontraron videos. Sube tus videos a Drive primero.")
elif videos_with_segments == 0 and videos_without_segments > 0 and len(segment_dirs) == 0:
    print("🎯 DETECTADO: Todos los archivos .mp4 son segmentos individuales.")
    print("   El sistema procesará cada archivo como un segmento independiente.")
    print("   No se requieren carpetas *_senas ni videos completos.")
    segments_dir = VIDEO_DIR
elif len(segment_dirs) > 0 and empty_dirs == len(segment_dirs) and len(segmented_groups) == 0:
    print("⚠️  DETECTADO: Todas las carpetas *_senas están vacías.")
    print("   El sistema buscará automáticamente videos pre-segmentados.")
    print("   Asegúrate de que tus segmentos estén en el mismo directorio que los videos")
    print("   con nombres como: NOMBRE_001.mp4, NOMBRE_002.mp4, etc.")
    segments_dir = VIDEO_DIR
elif len(segment_dirs) > 0:
    print("✅ Detectadas carpetas de segmentos tradicionales. Usando estructura de carpetas...")
    segments_dir = VIDEO_DIR
elif len(segmented_groups) > 0:
    print("✅ Detectados videos pre-segmentados. Procesando directamente...")
    segments_dir = VIDEO_DIR
else:
    print("❌ ERROR: No se encontraron segmentos ni videos pre-segmentados.")
    print("   Asegúrate de subir las carpetas *_senas o videos segmentados con patrón nombre_001.mp4, nombre_002.mp4, etc.")

if video_count > 0 and (len(segment_dirs) > 0 or len(segmented_groups) > 0 or (videos_with_segments == 0 and videos_without_segments > 0)):
    print("✅ Estructura correcta detectada. Procediendo con preparación de anotaciones...")

    # Crear anotaciones temporales usando el script optimizado
    !python scripts/prepare_annotations_fast.py \
        --videos_dir {VIDEO_DIR} \
        --segments_dir {segments_dir} \
        --output_dir data/annotations \
        --train_ratio 0.7 \
        --val_ratio 0.15 \
        --test_ratio 0.15

    print("✅ Anotaciones preparadas")

## ⚙️ 8. Configuración del Modelo

Ajustar configuración para Colab

In [ ]:
# Cargar y modificar configuración
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Ajustes para Colab T4 GPU (16GB VRAM)
config['data']['videos_dir'] = VIDEO_DIR
config['training']['batch_size'] = 4  # Ajustar según memoria disponible
config['training']['num_epochs'] = 100  # Puedes reducir para pruebas
config['training']['num_workers'] = 2  # Colab tiene 2 CPUs
config['training']['checkpoint_dir'] = CHECKPOINT_DIR
config['training']['log_dir'] = LOG_DIR

# Activar mixed precision para T4
config['training']['mixed_precision'] = True  # FP16 para mejor rendimiento

# Guardar configuración modificada
with open('config_colab.yaml', 'w') as f:
    yaml.dump(config, f, default_flow_style=False)

print("✅ Configuración ajustada para Colab T4 GPU")
print(f"   - Batch size: {config['training']['batch_size']}")
print(f"   - Epochs: {config['training']['num_epochs']}")
print(f"   - Mixed Precision: {config['training']['mixed_precision']}")

## 🚀 9. Entrenamiento

### Opción A: Entrenar desde cero

In [ ]:
# Entrenar desde cero
!python scripts/train.py \
    --config config_colab.yaml \
    --experiment_name "lsb_tad_v1"

print("✅ Entrenamiento completado")

### Opción B: Reanudar entrenamiento

In [ ]:
# Reanudar desde checkpoint (útil si Colab se desconecta)
CHECKPOINT_PATH = f"{CHECKPOINT_DIR}/epoch_50.pth"  # Ajustar al último checkpoint

!python scripts/train.py \
    --config config_colab.yaml \
    --resume {CHECKPOINT_PATH}

print("✅ Entrenamiento reanudado")

## 📊 10. Monitoreo con TensorBoard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir {LOG_DIR}

## 🎯 11. Evaluación

In [ ]:
# Evaluar modelo en test set
!python scripts/evaluate.py \
    --config config_colab.yaml \
    --checkpoint {CHECKPOINT_DIR}/best_model.pth \
    --split test \
    --output_dir results

# Mostrar métricas
import json
with open('results/metrics.json', 'r') as f:
    metrics = json.load(f)

print("\n" + "="*60)
print("RESULTADOS FINALES")
print("="*60)
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")
print("="*60)

## 🔮 12. Inferencia en Video Nuevo

In [ ]:
# Inferencia en un video
import torch
from models.complete_model import build_model
from utils.video_utils import load_video
from utils.visualization import visualize_detections
import cv2
from IPython.display import Video, display

# Cargar modelo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = build_model(config)
checkpoint = torch.load(f"{CHECKPOINT_DIR}/best_model.pth", map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)
model.eval()

print("✅ Modelo cargado")

# Procesar video
VIDEO_PATH = f"{VIDEO_DIR}/test_video.mp4"  # Ajustar ruta

with torch.no_grad():
    # Cargar video
    frames = load_video(VIDEO_PATH, num_frames=64)
    frames = frames.unsqueeze(0).to(device)  # (1, 3, T, H, W)
    
    # Inferencia
    outputs = model(frames)
    
    # Post-procesamiento
    detections = outputs['detections']  # [(start, end, class, score), ...]
    
    print(f"\n🎯 Detectadas {len(detections)} señas:")
    for i, det in enumerate(detections):
        start, end, class_id, score = det
        print(f"  {i+1}. Seña {class_id}: frames {start}-{end} (confianza: {score:.3f})")

# Visualizar resultados
output_path = "/content/output_visualized.mp4"
visualize_detections(VIDEO_PATH, detections, output_path)

# Mostrar video con detecciones
display(Video(output_path, width=640, embed=True))

## 📦 13. Inferencia en Lote

In [ ]:
# Procesar múltiples videos
import glob
from tqdm import tqdm

input_dir = f"{VIDEO_DIR}/test_videos"
output_dir = "/content/predictions"
os.makedirs(output_dir, exist_ok=True)

video_files = glob.glob(f"{input_dir}/*.mp4")

results = {}

for video_path in tqdm(video_files, desc="Procesando videos"):
    video_name = Path(video_path).stem
    
    with torch.no_grad():
        frames = load_video(video_path, num_frames=64)
        frames = frames.unsqueeze(0).to(device)
        
        outputs = model(frames)
        detections = outputs['detections']
        
        results[video_name] = detections

# Guardar resultados
import pickle
with open(f"{output_dir}/batch_predictions.pkl", 'wb') as f:
    pickle.dump(results, f)

print(f"✅ {len(results)} videos procesados")
print(f"   Resultados guardados en: {output_dir}/batch_predictions.pkl")

## 💾 14. Exportar Modelo

In [ ]:
# Exportar a ONNX para producción
dummy_input = torch.randn(1, 3, 64, 224, 224).to(device)

torch.onnx.export(
    model,
    dummy_input,
    f"{CHECKPOINT_DIR}/model.onnx",
    export_params=True,
    opset_version=14,
    do_constant_folding=True,
    input_names=['video'],
    output_names=['detections'],
    dynamic_axes={
        'video': {0: 'batch_size'},
        'detections': {0: 'batch_size'}
    }
)

print(f"✅ Modelo exportado a ONNX: {CHECKPOINT_DIR}/model.onnx")

## 📥 15. Descargar Resultados

In [ ]:
# Comprimir resultados para descarga
from google.colab import files
import shutil

# Crear ZIP con resultados importantes
shutil.make_archive('/content/lsb_tad_results', 'zip', '.', 
                    base_dir=None)

# Agregar a ZIP:
!zip -r /content/lsb_tad_results.zip \
    {CHECKPOINT_DIR}/best_model.pth \
    {CHECKPOINT_DIR}/model.onnx \
    results/ \
    config_colab.yaml

# Descargar
files.download('/content/lsb_tad_results.zip')

print("✅ Descarga iniciada")

## 💡 16. Tips y Troubleshooting

### ⚠️ Si te quedas sin memoria (CUDA OOM):
```python
# Reducir batch size
config['training']['batch_size'] = 2

# Reducir resolución de entrada
config['data']['input_size'] = 192  # en vez de 224

# Reducir longitud de clips
config['data']['clip_length'] = 32  # en vez de 64
```

### 🔄 Si Colab se desconecta:
- Los checkpoints están en Google Drive (si usaste `USE_DRIVE=True`)
- Simplemente ejecuta la celda de "Reanudar entrenamiento"

### 📊 Monitorear uso de GPU:
```python
!watch -n 1 nvidia-smi
```

### 🚀 Acelerar entrenamiento:
1. **Mixed Precision (FP16)**: Ya activado en config
2. **Gradient Accumulation**: Si batch size es muy pequeño
3. **DataLoader workers**: Ajustar `num_workers`

### 📈 Mejorar precisión:
1. Aumentar epochs
2. Usar data augmentation más agresivo
3. Ajustar learning rate
4. Usar learning rate scheduler

## 📈 17. Monitoreo en Tiempo Real

In [ ]:
# Monitorear progreso de entrenamiento
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import clear_output
import time

def plot_training_progress(log_file):
    """Graficar progreso en tiempo real."""
    try:
        df = pd.read_csv(log_file)
        
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # Loss
        axes[0, 0].plot(df['epoch'], df['train_loss'], label='Train')
        axes[0, 0].plot(df['epoch'], df['val_loss'], label='Val')
        axes[0, 0].set_title('Loss')
        axes[0, 0].set_xlabel('Epoch')
        axes[0, 0].legend()
        axes[0, 0].grid(True)
        
        # mAP
        axes[0, 1].plot(df['epoch'], df['val_map'])
        axes[0, 1].set_title('mAP@0.5')
        axes[0, 1].set_xlabel('Epoch')
        axes[0, 1].grid(True)
        
        # Learning Rate
        axes[1, 0].plot(df['epoch'], df['lr'])
        axes[1, 0].set_title('Learning Rate')
        axes[1, 0].set_xlabel('Epoch')
        axes[1, 0].set_yscale('log')
        axes[1, 0].grid(True)
        
        # GPU Memory
        if 'gpu_memory' in df.columns:
            axes[1, 1].plot(df['epoch'], df['gpu_memory'])
            axes[1, 1].set_title('GPU Memory (GB)')
            axes[1, 1].set_xlabel('Epoch')
            axes[1, 1].grid(True)
        
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"Error al graficar: {e}")

# Actualizar cada 30 segundos durante entrenamiento
LOG_FILE = f"{LOG_DIR}/training.csv"

# Descomentar para monitoreo en tiempo real (ejecutar en celda separada)
# while True:
#     clear_output(wait=True)
#     plot_training_progress(LOG_FILE)
#     time.sleep(30)

## 🧹 18. Limpieza (Opcional)

In [ ]:
# Limpiar archivos temporales para liberar espacio
!rm -rf /tmp/*
!rm -rf ~/.cache/pip
!rm -rf data/cache/*

# Ver espacio disponible
!df -h /content

print("✅ Limpieza completada")

---

## 🎓 Recursos Adicionales

- **Repositorio**: https://github.com/borysinho/LSB-Temporal-Action-Detection
- **Paper Video Swin**: [Video Swin Transformer](https://arxiv.org/abs/2106.13230)
- **Documentación PyTorch**: https://pytorch.org/docs/

---

## 📞 Soporte

¿Problemas o preguntas? Abre un issue en GitHub.

---

**Creado con ❤️ para la comunidad LSB**